In [1]:
from data_loading.data_loader import load_data

In [2]:
data = load_data()

In [3]:
import torch
import numpy as np

In [4]:
X = []
y = []
for d in data:
    X.append(d[0])
    y.append(d[1])
VAL_SPLIT = 0.25
VAL_SPLIT = len(X) * VAL_SPLIT
VAL_SPLIT = int(VAL_SPLIT)
X_train = X[:-VAL_SPLIT]
y_train = y[:-VAL_SPLIT]
X_test = X[-VAL_SPLIT:]
y_test = y[-VAL_SPLIT:]
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [5]:
from torchvision import models

In [6]:
device = torch.device('cuda')

In [7]:
import torch.nn as nn

In [8]:
model = models.resnet18(pretrained=True).to(device)
inf = model.fc.in_features
model.fc = nn.Linear(inf,2)

In [9]:
model = model.to(device)

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [12]:
BATCH_SIZE = 32

In [13]:
EPOCHS = 100

In [14]:
import wandb

In [15]:
PROJECT_NAME = 'Face-Mask-Detection'

In [16]:
from tqdm import tqdm

In [17]:
wandb.init(project=PROJECT_NAME,name='test')
for _ in tqdm(range(EPOCHS),leave=False):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        preds = model(X_batch)
        preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()